In [23]:
import pandas as pd
positivefile = open('reviews.pos','r',encoding='utf-8')
positives = pd.DataFrame({"reviews": positivefile.readlines(), "label": "pos"})
positives = pd.DataFrame(positives)

In [24]:
negativefile = open('reviews.neg','r',encoding='utf-8')
negatives = pd.DataFrame({"reviews": negativefile.readlines(), "label": "neg"})
negatives = pd.DataFrame(negatives)

In [55]:
negatives

,reviews,label
0,beklentimin altında bir ürün kaliteli değil\n,neg
1,3. kademe hız da motor titreme yapıyor.bu sebe...,neg
2,başlığı sabit durmuyor. arka koruma demiri üfl...,neg
3,ürün gayet kalitesiz öğrenci evinde aldığımız ...,neg
4,yuksek hizda kullanirken titredim yaptigi icin...,neg
...,...,...
14876,odada bu buhar makinası çalışırken uyumak kesi...,neg
14877,ürün gayet güzel ve sağlam.tek kusuru sesli ça...,neg
14878,hepsiburadaya tesekkurler\n,neg
14879,çok ufak ve basit plastikten yapılmış. hiç kul...,neg


In [56]:
reviews = pd.concat([positives[:15000], negatives], ignore_index=True)


In [57]:
reviews

,reviews,label
0,fena değil paraya göre iyi.\n,pos
1,ürün kaliteli ve çok kullanışlı kargo ya veril...,pos
2,fiyat-performans karşılaştırması yaptığınızda ...,pos
3,bu fiyata çok iyi\n,pos
4,fiyat ve performans olarak gayet iyi\n,pos
...,...,...
29876,odada bu buhar makinası çalışırken uyumak kesi...,neg
29877,ürün gayet güzel ve sağlam.tek kusuru sesli ça...,neg
29878,hepsiburadaya tesekkurler\n,neg
29879,çok ufak ve basit plastikten yapılmış. hiç kul...,neg


In [58]:
reviews['label'].value_counts()

label
pos    15000
neg    14881
Name: count, dtype: int64

In [59]:
import string
import re
import nltk
from nltk.corpus import stopwords

noktalama = string.punctuation
etkisiz = stopwords.words('turkish')

In [60]:
for d in reviews['reviews'].head():
    print(d+'\n-----------------')
    temp = ''
    for word in d.split():
        if word not in etkisiz and not word.isnumeric():
            temp+= word + ' '
    print(temp + '\n****************')

fena değil paraya göre iyi.

-----------------
fena değil paraya göre iyi. 
****************
ürün kaliteli ve çok kullanışlı kargo ya verilmesi ve elime çabuk ulaşması da gayet başarılı idi

-----------------
ürün kaliteli kullanışlı kargo verilmesi elime çabuk ulaşması gayet başarılı idi 
****************
fiyat-performans karşılaştırması yaptığınızda gayet iyi bir ürün. en önemlisi ısınma yapmıyor. 

-----------------
fiyat-performans karşılaştırması yaptığınızda gayet iyi bir ürün. önemlisi ısınma yapmıyor. 
****************
bu fiyata çok iyi

-----------------
fiyata iyi 
****************
fiyat ve performans olarak gayet iyi

-----------------
fiyat performans olarak gayet iyi 
****************


In [61]:
for d in reviews['reviews'].head():
    print(d+'\n------------')
    temp=''
    for word in d:
        if word not in noktalama:
            temp+=word
    print(temp+'\n************')
    d=temp

fena değil paraya göre iyi.

------------
fena değil paraya göre iyi

************
ürün kaliteli ve çok kullanışlı kargo ya verilmesi ve elime çabuk ulaşması da gayet başarılı idi

------------
ürün kaliteli ve çok kullanışlı kargo ya verilmesi ve elime çabuk ulaşması da gayet başarılı idi

************
fiyat-performans karşılaştırması yaptığınızda gayet iyi bir ürün. en önemlisi ısınma yapmıyor. 

------------
fiyatperformans karşılaştırması yaptığınızda gayet iyi bir ürün en önemlisi ısınma yapmıyor 

************
bu fiyata çok iyi

------------
bu fiyata çok iyi

************
fiyat ve performans olarak gayet iyi

------------
fiyat ve performans olarak gayet iyi

************


In [62]:
reviews.to_csv(r'./cleaned.csv',index=False)

In [63]:
import pandas as pd
data = pd.read_csv('cleaned.csv',sep=',',names=['review','label'])
print(data.head())

                                              review  label
0                                            reviews  label
1                      fena değil paraya göre iyi.\n    pos
2  ürün kaliteli ve çok kullanışlı kargo ya veril...    pos
3  fiyat-performans karşılaştırması yaptığınızda ...    pos
4                                bu fiyata çok iyi\n    pos


In [64]:
from sklearn.model_selection import train_test_split

In [65]:
X_train,X_test,y_train,y_test = train_test_split(data['review'].values.astype('U'),data['label'].values.astype('U'),test_size=0.1,random_state=42)

In [66]:
X_train.shape

(26893,)

In [67]:
X_test.shape

(2989,)

In [68]:
from sklearn.feature_extraction.text import CountVectorizer

In [69]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
print(X_train_counts.shape)

(26893, 55650)


In [70]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print(X_train_tfidf.shape)

(26893, 55650)


In [71]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf,y_train)
X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

In [72]:
y_pred = clf.predict(X_test_tfidf)
for review , sentiment in zip(X_test[:5],y_pred[:]):
    print('%r => %s' % (review,sentiment))

'ürünü aldım fiyatına göre bi ürün ve bu fiyata da bu ürün iyi sayılır. ürün hakkında kısa bilgi şöle olur. normal paspas gibi 2-3 mmlik bi kalınllıgı yok ama 2-3 mmlik kabartmalı bi ürün. ürün kalınlıgı ise 1mm dir. 2mm ye hayatta ulasmaz. ama alındıgında pişman olunacak bi ürün degil tavsiye ederim\n' => neg
'android 5 kurulu şarjı çok çabuk bitiyor.  isınma problemi çok fazla hadi bunları geçtim.  rehber donuyor.  arayanın ismi sonradan çıkıyor kişilere giriyorsunuz artık ne zaman açarsa ekran donuyor.\n' => neg
"ürünü çok iyi diye aldım fakat çek'im gücü azaldı ilk başlardaki gibi degil kullanalı iki ay oldu.\n" => neg
'abi bu fiyata kaliteli bisey diye aldım ama sivilce yapıyor güneş kremiyle uzaktan yakından alakası yok....\n' => neg
'duvarda denedim, hemen geri sökmeme rağmen boyayla birlikte ayrıldı. kapı dolap gibi tahta malzemelerde iyi yapışıyor. atölyelerde alet edavat yapıştırılabilir. gerçekten tekrar kullanılabiliyor. duvara birşeyler yapıştırmak için almayın derim.  \n'

In [73]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

0.8782201405152225


In [89]:
def öğren(mesaj):
    X_test = [mesaj]
    X_test_counts = count_vect.transform(X_test)
    X_test_tfidf = tfidf_transformer.transform(X_test_counts)
    y_pred = clf.predict(X_test_tfidf)
    if y_pred[0] == 'pos': 
        return 'Pozitif'
    else: 
        return 'Negatif'

In [92]:
öğren('Çok memnun kaldım')

['pos']


'Pozitif'

In [93]:
öğren('Rezalet resmen')

['neg']


'Negatif'

In [94]:
öğren('Güzel ürün iyi kargolama')

['pos']


'Pozitif'

In [95]:
öğren('tavsiye ederim')

['pos']


'Pozitif'